In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
 
batch_size = 100
 
train_dataset = datasets.MNIST(root='./mnist_data/',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)
 
test_dataset = datasets.MNIST(root='./mnist_data/',
                             train=False,
                             transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

0it [00:00, ?it/s]

9920512it [00:02, 4383859.00it/s]                             


Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 58504.43it/s]                           
0it [00:00, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw


1654784it [00:01, 967101.53it/s]                             
0it [00:00, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw


8192it [00:00, 21742.57it/s]            

Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw
Processing...
Done!


In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.relu = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm2d(num_features=64)

        self.conv1 = nn.Conv2d(in_channels=1, out_channels = 64, kernel_size=5, stride = 1, padding=2)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels = 64, kernel_size=5, stride = 2)
        self.dropout1 = nn.Dropout(0.25)
        self.FC1 = nn.Linear(64 *12 * 12, 128)
        self.dropout2 = nn.Dropout(0.5)
        self.FC2 = nn.Linear(128, 10)
        
    def forward(self, x):
        in_size = x.size(0)
        
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.view(in_size, -1)
        x = self.dropout1(x)
        x = self.relu(self.FC1(x))
        x = self.dropout2(x)
        x = self.FC2(x)
        # softmax?

        return x

In [0]:
model = Net()
 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        
        test_loss += criterion(output, target).item()
        
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [18]:
for epoch in range(1, 10):
    train(epoch)	
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303081
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.295185
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.295088
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.282523
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.258262
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.199879
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.111128
Train Epoch: 1 [7000/60000 (12%)]	Loss: 1.952731
Train Epoch: 1 [8000/60000 (13%)]	Loss: 1.664795
Train Epoch: 1 [9000/60000 (15%)]	Loss: 1.249524
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.979435
Train Epoch: 1 [11000/60000 (18%)]	Loss: 0.984675
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.785377
Train Epoch: 1 [13000/60000 (22%)]	Loss: 0.882440
Train Epoch: 1 [14000/60000 (23%)]	Loss: 0.647004
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.697457
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.779328
Train Epoch: 1 [17000/60000 (28%)]	Loss: 0.547193
Train Epoch: 1 [18000/60000 (30%)]	Loss: 0.550872
Train Epoch: 1 [19000/60000 (32%)]	Loss: 0.489352
Train Epoch: 1 [200

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Test set: Average loss: 0.0023, Accuracy: 9321/10000 (93%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.430739
Train Epoch: 2 [1000/60000 (2%)]	Loss: 0.240855
Train Epoch: 2 [2000/60000 (3%)]	Loss: 0.249093
Train Epoch: 2 [3000/60000 (5%)]	Loss: 0.476888
Train Epoch: 2 [4000/60000 (7%)]	Loss: 0.412180
Train Epoch: 2 [5000/60000 (8%)]	Loss: 0.319898
Train Epoch: 2 [6000/60000 (10%)]	Loss: 0.292202
Train Epoch: 2 [7000/60000 (12%)]	Loss: 0.359382
Train Epoch: 2 [8000/60000 (13%)]	Loss: 0.299904
Train Epoch: 2 [9000/60000 (15%)]	Loss: 0.381389
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.316997
Train Epoch: 2 [11000/60000 (18%)]	Loss: 0.282765
Train Epoch: 2 [12000/60000 (20%)]	Loss: 0.425729
Train Epoch: 2 [13000/60000 (22%)]	Loss: 0.249325
Train Epoch: 2 [14000/60000 (23%)]	Loss: 0.351684
Train Epoch: 2 [15000/60000 (25%)]	Loss: 0.296409
Train Epoch: 2 [16000/60000 (27%)]	Loss: 0.397972
Train Epoch: 2 [17000/60000 (28%)]	Loss: 0.172038
Train Epoch: 2 [18000/60000 (30%)]	Loss: 0.347971
Train Ep